## Model Selection

This notebook should include preliminary and baseline modeling.
- Try as many different models as possible.
- Don't worry about hyperparameter tuning or cross validation here.
- Ideas include:
    - linear regression
    - support vector machines
    - random forest
    - xgboost

In [139]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np



In [140]:
X_train = pd.read_csv("../data/processed/X_train_scaled.csv")
X_test = pd.read_csv("../data/processed/X_test_scaled.csv")
y_train = pd.read_csv("../data/processed/y_train.csv").squeeze()  # Convert to Series
y_test = pd.read_csv("../data/processed/y_test.csv").squeeze()


In [141]:
columns_to_drop = ["is_foreclosure", "is_price_reduced", "is_new_listing", "sub_type"]
X_train = X_train.drop(columns = columns_to_drop)
X_test = X_test.drop(columns = columns_to_drop)

# Create dummy variables from the "type" column
type_dummies = pd.get_dummies(X_train["type"], prefix="type", drop_first=True)

# Drop the original "type" column and concatenate the dummy variables
X_train = pd.concat([X_train.drop("type", axis=1), type_dummies], axis=1)
X_test = pd.concat([X_test.drop("type", axis=1), type_dummies], axis=1)

columns_with_nans = ["stories", "year_built", "latitude", "longitude"]

for col in columns_with_nans:
    X_train[col] = X_train[col].fillna(X_train[col].median())

for col in columns_with_nans:
    X_test[col] = X_test[col].fillna(X_test[col].median())

X_train = X_train.drop(columns=["list_date"])
X_train = X_train.dropna(subset=["city_encoded"])

X_test = X_test.drop(columns=["list_date"])
X_test = X_test.dropna(subset=["city_encoded"])

# Save cleaned version (optional)
X_train.to_csv("../data/processed/X_train_scaled_cleaned.csv", index=False)
X_test.to_csv("../data/processed/X_test_scaled_cleaned.csv", index=False)

In [142]:
y_train = y_train.loc[X_train.index]
y_test = y_test.loc[X_test.index]

y_train.to_csv("../data/processed/y_train_cleaned.csv", index=False)
y_test.to_csv("../data/processed/y_test_cleaned.csv", index=False)

In [143]:
print(X_train.dtypes[X_train.dtypes == "object"])
#X_train
#X_train.isna().sum()

Series([], dtype: object)


In [144]:
def evaluate_model(model, X_test, y_test):
    preds = model.predict(X_test)
    rmse = mean_squared_error(y_test, preds, squared=False)
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    return {"RMSE": rmse, "MAE": mae, "R2": r2}


In [145]:
models = {
    "Linear Regression": LinearRegression(),
    "Support Vector Regressor": SVR(),
    "Random Forest": RandomForestRegressor(random_state=41),
    "XGBoost": XGBRegressor(random_state=41),
    "ExtraTreesRegressor": ExtraTreesRegressor(random_state=42),
    "Ridge/Lasso Regression": Ridge(alpha=1.0)

}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_test, y_test)


To evaluate the success of models predicting house prices, the most effective metrics are Root Mean Squared Error (RMSE), Mean Absolute Error (MAE), and the R² score. These three collectively provide a holistic view of performance. RMSE reflects how far off predictions are from actual values on average (in the same units as the target variable), MAE gives a simple average of error magnitudes regardless of direction, and R² indicates how much of the variance in the target variable is explained by the model. Together, they help assess both prediction accuracy and model reliability.

Mean Squared Error (MSE) by itself is not ideal for real-world interpretation because it expresses the average error in squared units of the target, which in this case would be dollars squared. As a result, it's difficult for stakeholders to grasp what an MSE of, say, 1,000,000 actually means in practical terms. While MSE is useful for optimization during model training, it lacks intuitive meaning for reporting or comparing models.

This is why RMSE is preferable—it takes the square root of MSE and converts the error back into the original unit, making it easier to understand. For example, an RMSE of 70,000 means that, on average, the model's predictions deviate from the actual sale price by $70,000. This allows data scientists and business users alike to make sense of the model's performance in practical terms.

However, RMSE is very sensitive to outliers. Because it squares the error before averaging, any large errors have a disproportionately large effect on the final value. This means that even if the majority of predictions are accurate, just a few extreme outliers can significantly increase the RMSE. This property can be useful if large prediction errors are especially undesirable, but it can also be misleading in datasets with natural variance or noise. For this dataset, outliers were eliminated.

MAE is a more robust metric in the presence of outliers. By averaging the absolute values of the errors without squaring them, it gives equal weight to all deviations regardless of size. This makes MAE particularly useful when the goal is to understand the typical error in a dataset with a mix of small and large deviations, and when outliers shouldn’t overly influence model evaluation.

R², or the coefficient of determination, measures the proportion of variance in the dependent variable that the model can explain. A high R² score indicates that the model is good at capturing the underlying trend of the data. However, R² always increases as more features are added to the model—even if those features are irrelevant. This makes it a potentially misleading metric when comparing models with different numbers of predictors.

To address this limitation, adjusted R² is used. Unlike R², adjusted R² takes into account the number of independent variables and penalizes the model for adding features that don’t improve predictive power. This makes it particularly helpful during feature selection and when deciding whether a more complex model is justified. Even if the performance remains the same, a simpler model with a similar adjusted R² is generally preferable due to its interpretability and reduced risk of overfitting.

In summary, RMSE helps translate model performance into real-world financial terms, MAE offers a stable measure of average error unaffected by outliers, and R² (along with adjusted R²) provides a sense of how well the model captures the data’s variability. Together, these metrics offer a balanced and interpretable framework for evaluating and comparing regression models.


In [146]:
# gather evaluation metrics and compare results

results_df = pd.DataFrame(results).T  # Transpose so models are rows
print(results_df.sort_values("RMSE"))  # Sort by RMSE (best to worst)

                                   RMSE            MAE        R2
XGBoost                    71066.752811   33205.092521  0.931205
Random Forest              86408.857850   28173.138858  0.898295
ExtraTreesRegressor        89705.040790   33360.109530  0.890387
Ridge/Lasso Regression    170480.489028  109075.948380  0.604108
Linear Regression         170553.613213  109095.155286  0.603769
Support Vector Regressor  275198.974802  186455.771427 -0.031622


The results show that XGBoost performed the best overall, with the lowest RMSE - $71,067, a relatively low MAE - $33,205, and a high R² score of 0.93, meaning it explains 93% of the variance in the target variable. Random Forest also performed well, slightly behind XGBoost, with an R² of 0.90. ExtraTreesRegressor was close to Random Forest in performance but had a slightly higher error.

The Ridge/Lasso Regression and Linear Regression models performed significantly worse, with RMSEs over $170,000 and R² scores around 0.60, showing they couldn’t capture the complexity of the data well. Support Vector Regressor performed the worst, with a negative R² (meaning it did worse than simply predicting the average) and extremely high error values. This indicates that tree-based ensemble models like XGBoost and Random Forest are much more effective for this dataset.









## Feature Selection - STRETCH

> **This step doesn't need to be part of your Minimum Viable Product (MVP), but its recommended you complete it if you have time!**

Even with all the preprocessing we did in Notebook 1, you probably still have a lot of features. Are they all important for prediction?

Investigate some feature selection algorithms (Lasso, RFE, Forward/Backward Selection)
- Perform feature selection to get a reduced subset of your original features
- Refit your models with this reduced dimensionality - how does performance change on your chosen metrics?
- Based on this, should you include feature selection in your final pipeline? Explain

Remember, feature selection often doesn't directly improve performance, but if performance remains the same, a simpler model is often preferrable. 



In [ ]:
# perform feature selection 
# refit models
# gather evaluation metrics and compare to the previous step (full feature set)